In [ ]:
# pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# PyTorch: 2.7.1+cu118
# CUDA disponível: True
# CUDA version: 11.8
# GPU: NVIDIA GeForce GTX 1050 Ti
# Compute Capability: (6, 1)
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA disponível: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"Compute Capability: {torch.cuda.get_device_capability(0)}")
print(f"Arquiteturas suportadas: {torch.cuda.get_arch_list()}")

In [9]:
# Configurações globais do pandas para não truncar dados nos resultados

import pandas as pd

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)


In [4]:
from langchain_ollama import ChatOllama

BASE_URL = "http://localhost:11434"
MODEL = 'gemma3:4b'
# MODEL = 'gemma3:12b'
# MODEL = 'llama3.2:1b'
# model = 'llama3.2:1b'
# model = 'sheldon'
# model = 'sherlock'

# model = 'uncensored'

llm = ChatOllama(
    # URL do servidor Ollama
    base_url=BASE_URL,
    # Modelo de linguagem (LLaMA ou Gemma)
    model=MODEL,
    # ------------------------------------------------------------------
    # PARÂMETROS DE CRIATIVIDADE / ALEATORIEDADE
    # ------------------------------------------------------------------
    # Controla aleatoriedade (0.0 = determinístico)
    temperature=0.1,
    # Nucleus sampling
    top_p=0.9,
    # Top-K sampling
    top_k=40,
    # ------------------------------------------------------------------
    # CONTROLE DE REPETIÇÃO
    # ------------------------------------------------------------------
    # Penaliza repetições
    repeat_penalty=1.15,
    # Introdução de novos conceitos
    presence_penalty=0.0,
    # Penalização por frequência
    frequency_penalty=0.0,
    # ------------------------------------------------------------------
    # PARÂMETROS ESTRUTURAIS
    # ------------------------------------------------------------------
    # Janela de contexto
    num_ctx=4096,
    # Máx. tokens gerados
    num_predict=512,
    streaming=True,
)

In [ ]:
print(llm.invoke("O que significa a sigla RAG?").content)

In [ ]:
# Preparando os Dados (Documentos)
# Vamos começar com textos simples (isso vale igualmente para PDFs, Word, etc.).
# Em produção, esses textos viriam de arquivos ou banco de dados.

documentos = [
"RAG combina recuperação de informação com modelos de linguagem.",
"LLMs podem alucinar quando não possuem contexto externo.",
"FAISS é uma biblioteca eficiente para busca vetorial local.",
"Ollama permite rodar modelos LLM localmente via API."
]

In [ ]:
# Criando Embeddings (transformar texto em vetores)
# LLMs não fazem busca. Quem faz busca são embeddings.

from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
model_name="sentence-transformers/all-MiniLM-L6-v2"
)

In [ ]:
# Criando o Vector Store (FAISS)
# O FAISS guarda vetores e permite busca por similaridade.

from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_texts(
texts=documentos,
embedding=embeddings
)

In [ ]:
# Criando o Retriever (busca inteligente)
# O retriever define quantos documentos buscar.

retriever = vectorstore.as_retriever(
search_kwargs={"k": 3}
)

In [ ]:
# Criando o Prompt RAG (parte mais importante)
# O prompt obriga o modelo a usar o contexto.

from langchain_core.prompts import ChatPromptTemplate


prompt = ChatPromptTemplate.from_template(
"""
Você é um assistente técnico.
Use APENAS o contexto abaixo para responder.
Se a resposta não estiver no contexto, diga que não sabe.


Contexto:
{context}


Pergunta:
{question}
"""
)

In [ ]:
# Montando o RAG com LCEL (forma moderna)
# LCEL = LangChain Expression Language.
# A pergunta vira embedding
# O retriever busca contexto
# O LLM responde com base nele

from langchain_core.output_parsers import StrOutputParser

rag_chain = (
{
"context": retriever,
"question": lambda x: x
}
| prompt
| llm
| StrOutputParser()
)

In [ ]:
pergunta = "O que é alucinação no contexto LLM?"

resposta = rag_chain.invoke(pergunta)
print(resposta)

In [ ]:
from pypdf import PdfReader


def load_pdf(path):
    reader = PdfReader(path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
    return text


texto_pdf = load_pdf("datasets/Open-Set Tattoo Semantic Segmentation - Brilhador 2024.pdf")

In [ ]:
from docx import Document


def load_docx(path):
    doc = Document(path)
    return "\n".join([p.text for p in doc.paragraphs])


texto_docx = load_docx("documento.docx")

In [ ]:
# Chunking com LangChain

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
chunk_size=800,
chunk_overlap=120,
separators=[
"\n\n", # parágrafos
"\n", # quebras de linha
". ", # frases
" ",
""
]
)

In [15]:
"""
Tutorial Prático de RAG – Chunking correto de PDFs e Word (com overlap)
=====================================================================

Objetivo:
- Ler PDFs e DOCX
- Aplicar chunking correto com overlap
- Gerar Documents com metadados
- Criar embeddings
- Armazenar em FAISS (pronto para RAG)

Requisitos:
- Python 3.11
- Ollama já configurado (não usado diretamente aqui)
"""

# ============================================================
# 1. INSTALAÇÃO (executar uma única vez no ambiente)
# ============================================================
# pip install pypdf python-docx
# pip install langchain langchain-community langchain-text-splitters
# pip install sentence-transformers faiss-cpu


# ============================================================
# 2. IMPORTS
# ============================================================
import os 

from pypdf import PdfReader
from docx import Document

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document as LCDocument
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


# ============================================================
# 3. FUNÇÕES DE LEITURA DE ARQUIVOS
# ============================================================

def load_pdf(path: str) -> str:
    """
    Lê um arquivo PDF e retorna todo o texto como uma string.
    PDFs escaneados exigem OCR (não tratado aqui).
    """
    reader = PdfReader(path)
    text = ""

    for page in reader.pages:
        page_text = page.extract_text()
        if page_text:
            text += page_text + "\n"

    return text


def load_docx(path: str) -> str:
    """
    Lê um arquivo Word (.docx) e retorna o texto consolidado.
    """
    doc = Document(path)
    paragraphs = [p.text for p in doc.paragraphs if p.text.strip()]
    return "\n".join(paragraphs)


# ============================================================
# 4. CONFIGURAÇÃO DO CHUNKING (PARTE MAIS CRÍTICA DO RAG)
# ============================================================

# Parâmetros recomendados para modelos pequenos/médios
CHUNK_SIZE = 800        # tamanho do chunk em caracteres
CHUNK_OVERLAP = 120     # overlap (~15%)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    separators=[
        "\n\n",  # prioriza parágrafos
        "\n",
        ". ",
        " ",
        "",
    ],
)


# ============================================================
# 5. CARREGAMENTO DOS DOCUMENTOS
# ============================================================

# Ajuste os caminhos conforme necessário
pdf_text = load_pdf("datasets/tattoo.pdf")
# docx_text = load_docx("datasets/COBIT 4 para concursos - Bruno Marota.docx")


# ============================================================
# 6. GERAÇÃO DOS CHUNKS
# ============================================================

pdf_chunks = text_splitter.split_text(pdf_text)
# docx_chunks = text_splitter.split_text(docx_text)

print(f"Chunks PDF: {len(pdf_chunks)}")
# print(f"Chunks DOCX: {len(docx_chunks)}")


# ============================================================
# 7. CRIAÇÃO DE DOCUMENTS COM METADADOS
# ============================================================

documents = []

# Chunks do PDF
for i, chunk in enumerate(pdf_chunks):
    documents.append(
        LCDocument(
            page_content=chunk,
            metadata={
                "source": "documento.pdf",
                "chunk_id": i,
                "type": "pdf",
            },
        )
    )

# Chunks do DOCX
# for i, chunk in enumerate(docx_chunks):
#     documents.append(
#         LCDocument(
#             page_content=chunk,
#             metadata={
#                 "source": "documento.docx",
#                 "chunk_id": i,
#                 "type": "docx",
#             },
#         )
#     )

print(f"Total de Documents finais: {len(documents)}")


# ============================================================
# 8. EMBEDDINGS (TRANSFORMA TEXTO EM VETORES)
# ============================================================

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device": "cpu"},
    encode_kwargs={
        "normalize_embeddings": True,
        "batch_size": 32,
    },
)


# ============================================================
# 9. VECTOR STORE (FAISS)
# ============================================================

vectorstore = FAISS.from_documents(
    documents=documents,
    embedding=embeddings,
)

print("Vector store FAISS criado com sucesso.")

retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3},
)


# VECTORSTORE_DIR = "vectorstore_faiss"
# os.makedirs(VECTORSTORE_DIR, exist_ok=True)
# vectorstore.save_local(VECTORSTORE_DIR)
# print(f"Vector store salvo em: {VECTORSTORE_DIR}")


# ============================================================
# 9. PROMPT RAG (SUBSTITUI QUERY DIRETA)
# ============================================================

prompt = ChatPromptTemplate.from_template(
    """
Você é um assistente técnico.
Use APENAS o contexto abaixo para responder.
Se a resposta não estiver no contexto, diga que não sabe.

Contexto:
{context}

Pergunta:
{question}
"""
)


# ============================================================
# 11. FUNÇÃO AUXILIAR PARA FORMATAR CONTEXTO
# ============================================================

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# ============================================================
# 12. PIPELINE RAG (LCEL – PADRÃO MODERNO)
# ============================================================

rag_chain = (
    {
        "context": retriever | format_docs,
        "question": lambda x: x,
    }
    | prompt
    | llm
    | StrOutputParser()
)


# ============================================================
# 13. EXECUÇÃO
# ============================================================

pergunta = "Para que serve as tatuagens?"

resposta = rag_chain.invoke(pergunta)

print("\nResposta do RAG:\n")
print(resposta)


# ============================================================
# 14. OBSERVAÇÃO PROFISSIONAL
# ============================================================

"""
Este é o padrão correto de RAG:

- O usuário NÃO fala direto com o LLM
- O prompt controla o comportamento
- O contexto vem EXCLUSIVAMENTE do retriever
- Isso reduz alucinação e permite avaliação objetiva

Próximo passo natural:
- Avaliação com Recall@k
- Avaliação de Faithfulness (RAGAS)
- API FastAPI para produção
"""


Chunks PDF: 165
Total de Documents finais: 165
Vector store FAISS criado com sucesso.

Resposta do RAG:

Não sei.


'\nEste é o padrão correto de RAG:\n\n- O usuário NÃO fala direto com o LLM\n- O prompt controla o comportamento\n- O contexto vem EXCLUSIVAMENTE do retriever\n- Isso reduz alucinação e permite avaliação objetiva\n\nPróximo passo natural:\n- Avaliação com Recall@k\n- Avaliação de Faithfulness (RAGAS)\n- API FastAPI para produção\n'

In [8]:
# ============================================================
# 13. EXECUÇÃO
# ============================================================

pergunta = "Quem são os autores do artigo científico?"

resposta = rag_chain.invoke(pergunta)

print("\nResposta do RAG:\n")
print(resposta)


Resposta do RAG:

Não sei.


In [16]:
# ============================================================
# INSPECIONAR CHUNKS ARMAZENADOS NO VECTORSTORE FAISS
# ============================================================

# Objetivo:
# - Verificar quais chunks foram realmente salvos no FAISS
# - Inspecionar texto + metadados
# - Fazer debug de ingestão (etapa crítica em RAG)

# Premissas:
# - vectorstore já foi criado OU carregado do disco
# - embeddings compatíveis já estão configurados


# ------------------------------------------------------------
# 1. ACESSAR DOCUMENTOS INTERNOS DO VECTORSTORE
# ------------------------------------------------------------
# O FAISS guarda os Documents em um docstore interno

docstore = vectorstore.docstore
doc_ids = list(docstore._dict.keys())

print(f"Total de chunks no vectorstore: {len(doc_ids)}")


# ------------------------------------------------------------
# 2. VISUALIZAR ALGUNS CHUNKS (RECOMENDADO)
# ------------------------------------------------------------
# Evita poluir o terminal com texto demais

for doc_id in doc_ids[:]:  # mostra apenas os 5 primeiros
    doc = docstore._dict[doc_id]

    print("=" * 80)
    print(f"Doc ID: {doc_id}")
    print(f"Source: {doc.metadata.get('source')}")
    print(f"File type: {doc.metadata.get('file_type')}")
    print(f"Chunk ID: {doc.metadata.get('chunk_id')}")
    print(f"Chunking: {doc.metadata.get('chunking')}")
    print(f"Tamanho (caracteres): {len(doc.page_content)}")
    print("-" * 80)
    print(doc.page_content[:])  # limita o texto exibido 600
    print("...\n")


# ------------------------------------------------------------
# 3. INSPEÇÃO COMPLETA (USAR COM CUIDADO)
# ------------------------------------------------------------
# Se você realmente quiser ver TODOS os chunks:

# for doc_id in doc_ids:
#     doc = docstore._dict[doc_id]
#     print(doc.page_content)


# ============================================================
# OBSERVAÇÃO PROFISSIONAL IMPORTANTE
# ============================================================

"""
Boas práticas ao inspecionar chunks:

✔ Sempre verifique os chunks ANTES de culpar o modelo
✔ Confira se não há chunks vazios ou duplicados
✔ Verifique tamanhos (<= 1000 chars)
✔ Confirme metadados (source, chunk_id)

90% dos problemas de RAG vêm de ingestão mal validada.
"""


Total de chunks no vectorstore: 165
Doc ID: 04c585af-e686-4517-af6d-5a41b4d30513
Source: documento.pdf
File type: None
Chunk ID: 0
Chunking: None
Tamanho (caracteres): 752
--------------------------------------------------------------------------------
Date of publication xxxx 00, 0000, date of current version xxxx 00, 0000.
Digital Object Identifier 10.1109/ACCESS.2017.DOI
Open-Set Tattoo Semantic Segmentation
ANDERSON BRILHADOR 1, RODRIGO TCHALSKI DA SIL VA 1, CARLOS ROBERTO
MODINEZ-JUNIOR 1, GABRIEL DE ALMEIDA SP ADAFORA 1, HEITOR SIL VÉRIO
LOPES 1, AND ANDRÉ EUGÊNIO LAZZARETTI 1, (Member, IEEE).
1Federal University of Technology - Paraná, Av. Sete de Setembro, 3165, Curitiba, 80230-901, Paraná, Brazil.
Corresponding author: Anderson Brilhador (e-mail: andersonbrilhador@gmail.com).
ABSTRACT Tattoos can serve as an essential source of biometric information for public security, aiding
in identifying suspects and victims. In order to automate tattoo classification, tasks like classific

'\nBoas práticas ao inspecionar chunks:\n\n✔ Sempre verifique os chunks ANTES de culpar o modelo\n✔ Confira se não há chunks vazios ou duplicados\n✔ Verifique tamanhos (<= 1000 chars)\n✔ Confirme metadados (source, chunk_id)\n\n90% dos problemas de RAG vêm de ingestão mal validada.\n'

In [11]:
"""
Chunking baseado em LLM (LLM-based Chunking)
===========================================

Objetivo:
- Usar um LLM local (Ollama) para dividir texto em chunks SEMÂNTICOS
- Garantir no máximo 1000 caracteres por chunk
- Ideal para textos técnicos, jurídicos e científicos

Observação importante:
- Isso é MAIS caro computacionalmente
- Porém gera chunks de MUITO maior qualidade semântica
"""

# ============================================================
# 1. IMPORTS
# ============================================================
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


# ============================================================
# 2. LLM LOCAL (OLLAMA)
# ============================================================

llm = ChatOllama(
    base_url="http://localhost:11434",
    model="gemma3:4b",
    temperature=0.0,   # chunking deve ser determinístico
)


# ============================================================
# 3. PROMPT DE CHUNKING SEMÂNTICO
# ============================================================

chunking_prompt = ChatPromptTemplate.from_template(
    """
Você é um especialista em organização de textos técnicos.

Tarefa:
- Divida o texto abaixo em CHUNKS SEMÂNTICOS.
- Cada chunk deve conter NO MÁXIMO 1000 caracteres.
- Nunca corte uma ideia no meio.
- Preserve a ordem original.
- Retorne os chunks separados pelo delimitador <CHUNK>.

Texto:
{text}
"""
)


# ============================================================
# 4. PIPELINE DE CHUNKING COM LLM
# ============================================================

chunking_chain = (
    chunking_prompt
    | llm
    | StrOutputParser()
)


# ============================================================
# 5. FUNÇÃO DE CHUNKING LLM-BASED
# ============================================================

def llm_based_chunking(text: str) -> list[str]:
    """
    Recebe um texto grande e retorna uma lista de chunks
    semanticamente coerentes com até 1000 caracteres.
    """
    response = chunking_chain.invoke({"text": text})

    # Divide usando o delimitador definido no prompt
    chunks = [
        chunk.strip()
        for chunk in response.split("<CHUNK>")
        if chunk.strip()
    ]

    # Garantia defensiva (fallback)
    for chunk in chunks:
        if len(chunk) > 1000:
            raise ValueError(
                "Chunk maior que 1000 caracteres detectado. "
                "Revise o prompt ou modelo."
            )

    return chunks


# ============================================================
# 6. EXEMPLO DE USO
# ============================================================

texto_exemplo = """
RAG (Retrieval-Augmented Generation) é uma técnica que combina
modelos de linguagem com mecanismos de recuperação de informação.
Ela é amplamente usada para reduzir alucinações e melhorar
respostas baseadas em conhecimento específico.
"""

chunks = llm_based_chunking(texto_exemplo)

for i, chunk in enumerate(chunks):
    print("=" * 80)
    print(f"CHUNK {i} | {len(chunk)} caracteres")
    print("-" * 80)
    print(chunk)
    print()


# ============================================================
# 7. OBSERVAÇÃO PROFISSIONAL
# ============================================================

"""
Quando usar LLM-based chunking?

✔ Textos longos e complexos
✔ Documentos jurídicos
✔ Manuais técnicos
✔ PDFs mal estruturados

Quando NÃO usar?

✘ Grandes volumes (milhares de documentos)
✘ Quando custo/latência importa mais que qualidade

Padrão profissional:
- Chunking heurístico (RecursiveCharacterTextSplitter) para 80%
- LLM-based chunking apenas para documentos críticos
"""



CHUNK 0 | 128 caracteres
--------------------------------------------------------------------------------
RAG (Retrieval-Augmented Generation) é uma técnica que combina
modelos de linguagem com mecanismos de recuperação de informação.

CHUNK 1 | 105 caracteres
--------------------------------------------------------------------------------
Ela é amplamente usada para reduzir alucinações e melhorar
respostas baseadas em conhecimento específico.



'\nQuando usar LLM-based chunking?\n\n✔ Textos longos e complexos\n✔ Documentos jurídicos\n✔ Manuais técnicos\n✔ PDFs mal estruturados\n\nQuando NÃO usar?\n\n✘ Grandes volumes (milhares de documentos)\n✘ Quando custo/latência importa mais que qualidade\n\nPadrão profissional:\n- Chunking heurístico (RecursiveCharacterTextSplitter) para 80%\n- LLM-based chunking apenas para documentos críticos\n'